## CNN Train Script

Este es el archivo donde se hará una prueba de clasificador automático mediante CNN (Convolutional Neural Network).

In [1]:
import os
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

Para puntualizar ahora mismo, se añadirán mas tipos en un futuro.
1. original_signals → Etiqueta 0.
2. flicker_signals → Etiqueta 1.
3. harmonic_signals → Etiqueta 2.
4. Interruption_signals → Etiqueta 3.
5. original_signals → Etiqueta 4.
6. Swell_signals → Etiqueta 5.
7. transient_signals → Etiqueta 6.

In [1]:
# Funciones para cargar la señal y extraer características
def load_signal(datos_path):
    data = scipy.io.loadmat(datos_path)
    signal = data['signal'][0]
    fs = data['Fs'][0, 0]
    return signal, fs

def signal_feature_extraction(signal, fs):
    max_theoretical_value = 230 * np.sqrt(2)
    normalized_signal = signal / max_theoretical_value
    mean_value = np.mean(normalized_signal)
    unbias_data = normalized_signal - mean_value
    unbias_data_2 = unbias_data ** 2
    unbias_data_3 = unbias_data_2 * unbias_data
    unbias_data_4 = unbias_data_3 * unbias_data
    variance = np.var(unbias_data)
    skewness = np.mean(unbias_data_3) / (variance ** 1.5)
    kurtosis = np.mean(unbias_data_4) / (variance ** 2) - 3
    thd = np.sqrt(np.sum(np.abs(np.fft.fft(normalized_signal)[2:4])) / np.abs(np.fft.fft(normalized_signal)[1]))
    rms = np.sqrt(np.mean(normalized_signal ** 2))
    crest_factor = np.max(normalized_signal) / rms
    return variance, skewness, kurtosis, thd, crest_factor

# Función para cargar datos desde carpetas
def load_data(base_dir):
    data = []
    labels = []
    classes = sorted(os.listdir(base_dir))  # Asegura el orden al asignar etiquetas
    
    for class_idx, class_name in enumerate(classes):
        class_dir = os.path.join(base_dir, class_name)
        
        for split in ['train', 'test', 'val']:
            split_dir = os.path.join(class_dir, split)
            for file in os.listdir(split_dir):
                file_path = os.path.join(split_dir, file)
                # Cargar características desde archivo (suponiendo un formato numpy)
                features = np.load(file_path)
                data.append(features)
                labels.append(class_idx)
    
    return np.array(data), np.array(labels)

# Ruta de los datos
base_dir = "data"

# Cargar datos
features, labels = load_data(base_dir)

# Dividir en sets de entrenamiento, test y validación
features_train, features_rest, labels_train, labels_rest = train_test_split(features, labels, test_size=0.3, random_state=42)
features_val, features_test, labels_val, labels_test = train_test_split(features_rest, labels_rest, test_size=0.5, random_state=42)

# Convertir etiquetas a one-hot encoding
labels_train = to_categorical(labels_train, num_classes=7)
labels_val = to_categorical(labels_val, num_classes=7)
labels_test = to_categorical(labels_test, num_classes=7)

# --------- SEPARACION, HAY QUE QUITARLO DESPUÉS ----------

# Recorrer carpetas y obtener características
data_path = 'ruta_a_tus_datos'  # Cambia esto por el path a tu carpeta 'data'
labels = []  # Aquí asigna las etiquetas de las carpetas, por ejemplo, 0 para 'normal', 1 para 'fault'
features = []  # Lista para almacenar características
targets = []   # Lista para almacenar etiquetas

for folder in os.listdir(data_path):
    folder_path = os.path.join(data_path, folder)
    if os.path.isdir(folder_path):
        label = 0 if folder == 'normal' else 1  # Ajusta según tu organización
        for file in os.listdir(folder_path):
            if file.endswith('.mat'):
                file_path = os.path.join(folder_path, file)
                signal, fs = load_signal(file_path)
                feature_vector = signal_feature_extraction(signal, fs)
                features.append(feature_vector)
                targets.append(label)

# Convertir a numpy arrays
features = np.array(features)
targets = np.array(targets)
# Normalización de características
scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(features_normalized, targets, test_size=0.3, random_state=42)

FileNotFoundError: [Errno 2] No such file or directory: 'ruta_a_tus_datos'

In [ ]:
# Definir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(features_train.shape[1],)),  # Entrada
    Dense(32, activation='relu'),  # Capa oculta
    Dense(7, activation='softmax')  # Capa de salida (7 clases)
])

# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  # One-hot encoded labels
              metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(features_train, labels_train,
                    validation_data=(features_val, labels_val),
                    epochs=50,
                    batch_size=32)

# Evaluar el modelo
loss, accuracy = model.evaluate(features_test, labels_test)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
# Gráfico de pérdida
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Gráfico de precisión
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()